GridSearch (choose parameter to try, lr, dropout)
early stopping

Deactivating callbacks can be especially useful when you do a parameter search (say with sklearn GridSearchCV). If, for instance, you use a callback for learning rate scheduling (e.g. via LRScheduler) and want to test its usefulness, you can compare the performance once with and once without the callback.

Implement confusion matrix


In [3]:
import os
import numpy as np
import scipy.io

In [4]:
mods = ['BPSK', 'DQPSK', 'GFSK', 'GMSK', 'OQPSK',
        'PAM4', 'PAM8', 'PSK8', 'QAM16', 'QAM64', 'QPSK']
class_num = len(mods)

In [5]:
def import_from_mat(mods, fpath):

    # import data of mat format
    data = scipy.io.loadmat(fpath)
    features = []
    labels = []
    for mod in mods:
        real = np.array(data[mod].real)
        imag = np.array(data[mod].imag)
        signal = np.concatenate([real, imag], axis=1)
        features.append(signal)
        labels.append(mods.index(mod) * np.ones([signal.shape[0], 1]))

    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    return features, labels

In [6]:
features, labels = import_from_mat(mods, 
                        "D:/Archive/0006/"
                        "batch100000_symbols128_sps8_baud1_snr5.dat")

In [7]:
features = features.astype(np.float32)
labels = labels.astype(np.int64)
np.random.shuffle(features)
np.random.shuffle(labels)

In [8]:
X = features[:1000]
y = labels[:1000].reshape(-1)

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [10]:
class Discriminator(nn.Module):
    """Define the model"""

    def __init__(self):
        super(Discriminator, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(2, 256, 3, padding=1),  # batch, 256, 1024
            nn.BatchNorm1d(256),
            nn.ReLU(),
            # nn.Dropout2d()
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(256, 80, 3, padding=1),  # batch, 80, 1024
            nn.BatchNorm1d(80),
            nn.ReLU(),
            # nn.Dropout2d()
        )
        self.fc1 = nn.Sequential(
            nn.Linear(80 * 1024, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(p=0.6)
        )
        self.fc2 = nn.Sequential(
            nn.Linear(256, class_num),
            nn.ReLU()
        )
        
    def forward(self, x, **kwargs):
        x = x.reshape((x.size(0), 2, -1))
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = F.softmax(x, dim=1)
        return x

In [11]:
from skorch import NeuralNetClassifier
from skorch.callbacks import ProgressBar, EpochScoring, PrintLog
from skorch.utils import data_from_dataset
import ipdb as pdb
from sklearn.metrics import accuracy_score, confusion_matrix

In [31]:
class Score_ConfusionMatrix(EpochScoring):
    def on_epoch_end(self, net, dataset_train, dataset_valid, **kwargs):
        
        EpochScoring.on_epoch_end(self, net, dataset_train, dataset_valid)
        
        X_test, y_test, _ = self.get_test_data(dataset_train, dataset_valid)
        y_pred = net.predict(X_test)
        cm = confusion_matrix(y_test, y_pred)
        
        history = net.history
        history.record("confusion_matrix", cm)
        

In [32]:
class Print_Score_(PrintLog):
    def on_epoch_end(self, net, **kwargs):
        PrintLog.on_epoch_end(self, net)
        history = net.history
        print(history[-1, "confusion_matrix"])
        
        

In [27]:
class ConfusionMatrix(PrintLog):
    def on_epoch_end(self, net, **kwargs):
        PrintLog.on_epoch_end(self, net)
        dataset_valid = kwargs['dataset_valid']
        """
        https://github.com/dnouri/skorch/blob/a246194
        /skorch/callbacks/scoring.py#L371
        """
        X_test, y_test = data_from_dataset(dataset_valid)
        y_pred = net.predict(X_test)
        cm = confusion_matrix(y_test, y_pred)
        
        history = net.history
        history.record("confusion_matrix", cm)
        print(cm)
        
        

In [20]:
# class TrainProcess(EpochScoring):
#     def on_epoch_end(
#         self,
#         net,
#         dataset_train,
#         dataset_valid,
#         **kwargs
#     ):
#         X_test, y_test, _ = self.get_test_data(
#             dataset_train,
#             dataset_valid
#         )
#         y_pred = net.predict(X_test)
#         cm = confusion_matrix(y_test, y_pred)
        
#         history = net.history
#         history.record("confusion_matrix", cm)

In [12]:
# class TrainProcess(ProgressBar):
#     def on_epoch_end(self, net, **kwargs):
#         self.pbar.close()
#         pdb.set_trace()
#         train_X = kwargs['dataset_train'].dataset.X
#         train_y = kwargs['dataset_train'].dataset.y
#         val_X = kwargs['dataset_valid'].dataset.X
#         val_y = kwargs['dataset_valid'].dataset.y
#         train_proba = net.predict_proba(train_X)
#         val_proba = net.predict_proba(val_X)
#         train_loss = net.criterion(
#             train_proba,
#             train_y
#         )
#         val_loss = net.criterion(
#             val_proba,
#             val_y
#         )
#         train_pred = net.predict(train_X)
#         val_pred = net.predict(val_X)
#         train_acc = accuracy_score(
#             train_pred,
#             train_y
#         )
#         val_acc = accuracy_score(
#             val_pred,
#             val_y
#         )
#         train_cm = confusion_matrix(train_pred, train_y)
#         val_cm = confusion_matrix(val_pred, val_y)
#         history = net.history
#         history.new_epoch()
#         history.record("loss_whole_train_set", train_loss)
#         history.record("loss_whole_valid_set", val_loss)
#         history.record("accuracy_whole_train_set", train_acc)
#         history.record("accuracy_whole_valid_set", val_acc)
#         history.record("cm_whole_train_set", train_cm)
#         history.record("cm_whole_valid_set", val_cm)
#         history.record("prediction_whole_train_set", train_pred)
#         history.record("prediction_whole_valid_set", valid_pred)
#         print()
#         # TODO cal train loss, valid loss, train_acc, valid_acc
#         # TODO Output confusion matrix 
#         # TODO Output predict and y

In [33]:
net = NeuralNetClassifier(
    Discriminator,
    max_epochs=20,
    lr=0.01,
    device='cuda',
    iterator_train__shuffle=True,
    iterator_valid__shuffle=True
)

score = Score_ConfusionMatrix(scoring="accuracy", lower_is_better=False)
pt = (keys_ignored="confusion_matrix")
net.set_params(callbacks__valid_acc=score)
net.set_params(callbacks__print_log=pt)

<class 'skorch.classifier.NeuralNetClassifier'>[uninitialized](
  module=<class '__main__.Discriminator'>,
)

In [28]:
net = NeuralNetClassifier(
    Discriminator,
    max_epochs=20,
    lr=0.01,
    device='cuda',
    iterator_train__shuffle=True,
    iterator_valid__shuffle=True
)

net.set_params(callbacks__print_log=ConfusionMatrix())

<class 'skorch.classifier.NeuralNetClassifier'>[uninitialized](
  module=<class '__main__.Discriminator'>,
)

In [34]:
net.fit(X, y)

  epoch    accuracy    train_loss    valid_loss     dur
-------  ----------  ------------  ------------  ------
      1      0.0833        2.4209        2.3993  0.2969
      2      0.0980        2.2404        2.3997  0.2869
      3      0.1029        2.0935        2.4009  0.2937
      4      0.1176        1.9497        2.3930  0.2765
      5      0.0882        1.7513        2.3963  0.2763
      6      0.0931        1.5763        2.3911  0.2813
      7      0.0833        1.4268        2.4017  0.2888
      8      0.0931        1.2954        2.3993  0.2812
      9      0.0784        1.2192        2.4020  0.2969
     10      0.0931        1.1092        2.3988  0.2969
     11      0.0735        1.0410        2.4038  0.2814
     12      0.0882        0.9531        2.4002  0.2813
     13      0.0931        0.8904        2.4080  0.2813
     14      0.0784        0.8466        2.4059  0.2813
     15      0.0637        0.7883        2.4108  0.2969
     16      0.0784        0.7332        2.4060 

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=Discriminator(
    (conv1): Sequential(
      (0): Conv1d(2, 256, kernel_size=(3,), stride=(1,), padding=(1,))
      (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (conv2): Sequential(
      (0): Conv1d(256, 80, kernel_size=(3,), stride=(1,), padding=(1,))
      (1): BatchNorm1d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (fc1): Sequential(
      (0): Linear(in_features=81920, out_features=256, bias=True)
      (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Dropout(p=0.6)
    )
    (fc2): Sequential(
      (0): Linear(in_features=256, out_features=11, bias=True)
      (1): ReLU()
    )
  ),
)

In [ ]:
a = kwargs['dataset_train'].dataset.y

In [35]:
net.history

[{'accuracy': 0.08333333333333333,
  'accuracy_best': True,
  'batches': [{'train_batch_size': 128, 'train_loss': 2.430835723876953},
   {'train_batch_size': 128, 'train_loss': 2.4024858474731445},
   {'train_batch_size': 128, 'train_loss': 2.4167110919952393},
   {'train_batch_size': 128, 'train_loss': 2.4250473976135254},
   {'train_batch_size': 128, 'train_loss': 2.417490243911743},
   {'train_batch_size': 128, 'train_loss': 2.4235451221466064},
   {'train_batch_size': 28, 'train_loss': 2.4623420238494873},
   {'valid_batch_size': 128, 'valid_loss': 2.399543046951294},
   {'valid_batch_size': 76, 'valid_loss': 2.3988399505615234}],
  'confusion_matrix': array([[ 0,  0,  0,  0,  0, 19,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0, 21,  0,  0,  0,  0,  0],
         [ 0,  1,  0,  0,  0, 17,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0, 19,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0, 18,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0, 16,  0,  0,  0,  0,  0],
       

In [29]:
np.sum(net.history[-3,'confusion_matrix'])

205

In [44]:
xxx = net.get_params()['callbacks__valid_acc']

In [46]:
dir(xxx)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_get_name',
 '_get_param_names',
 '_initialize_cache',
 '_is_best_score',
 '_record_score',
 '_scoring',
 'get_params',
 'get_test_data',
 'initialize',
 'lower_is_better',
 'name',
 'on_batch_begin',
 'on_batch_end',
 'on_epoch_begin',
 'on_epoch_end',
 'on_grad_computed',
 'on_train',
 'on_train_begin',
 'on_train_end',
 'scoring',
 'set_params',
 'target_extractor',
 'use_caching']

In [ ]:
net.set_params(callback__default_callbacks=None)

In [29]:
net.get_default_callbacks()

[('epoch_timer', <skorch.callbacks.logging.EpochTimer at 0x29e9e27be10>),
 ('train_loss', <skorch.callbacks.scoring.BatchScoring at 0x29e9e27bb70>),
 ('valid_loss', <skorch.callbacks.scoring.BatchScoring at 0x29e9e27bb38>),
 ('valid_acc', <skorch.callbacks.scoring.EpochScoring at 0x29e9e27bb00>),
 ('print_log', <skorch.callbacks.logging.PrintLog at 0x29e9e27bac8>)]